In [0]:
import pickle
import os
import time
import numpy as np
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.layers import BatchNormalization as BatchNorm
from google.colab import files as fd
from zipfile import ZipFile

Using TensorFlow backend.


In [0]:
def get_notes ():
  ''' carica le note dal file '''
  with open ('data/notes','rb') as f:
    notes = pickle.load(f)
  return notes, len(set(notes))

In [0]:
def prepara_sequenza (notes,pitchnames,n_vocab):
  ''' prepara le sequenza per la rete di input per la rete neurale '''
  # mappa
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
  lunghezza_sequenza = 100
  network_input = []
  for i in range(0, len(notes) - lunghezza_sequenza, 1):
    sequence_in = notes[i:i + lunghezza_sequenza]
    sequence_out = notes[i + lunghezza_sequenza]
    network_input.append([note_to_int[char] for char in sequence_in])
  normalized_inputs = np.reshape (network_input,(len (network_input),lunghezza_sequenza,1))
  normalized_inputs = normalized_inputs/float (n_vocab)
  return network_input,normalized_inputs

In [0]:
def create_network(network_input, n_vocab,weights = None,model = None):
  '''creazione rete neurale (carica i pesi delle connessioni)'''
  if model is None:
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
  if weights is not None:
    model.load_weights (weights)
  return model

In [0]:
def genera_note (model,network_input,pitchnames,n_vocab,l = 500):
  ''' Fa generare alla rete deurale delle note; il brano ha lunghezza l '''
  start = np.random.randint(0, len(network_input)-1)
  # mappa
  int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
  pattern = network_input[start]
  prediction_output = []
  for _ in range (l):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
  return prediction_output

In [0]:
def salva_in_midi (prediction_output,name = 'test'):
  """ converte gli output della rete neurale in midi files """
  offset = 0
  output_notes = []
  for pattern in prediction_output:
      # il pattern è un accordo
      if ('.' in pattern) or pattern.isdigit():
          notes_in_chord = pattern.split('.')
          notes = []
          for nota in notes_in_chord:
              new_note = note.Note(int(nota))
              new_note.storedInstrument = instrument.Piano()
              notes.append(new_note)
          new_chord = chord.Chord(notes)
          new_chord.offset = offset
          output_notes.append(new_chord)
      # il pattern è una nota
      else:
          new_note = note.Note(pattern)
          new_note.offset = offset
          new_note.storedInstrument = instrument.Piano()
          output_notes.append(new_note)
      # per non far accavallare le note
      offset += 0.5
  midi_stream = stream.Stream(output_notes)
  midi_stream.write('midi', fp=f'{name}.mid')

In [0]:
def scrivi_brani (model_name,lunghezza = 500,nome_files = ['test'],model = None,rm = False):
  notes,n_vocab = get_notes()
  pitchnames = sorted(set(item for item in notes))
  network_input, normalized_input = prepara_sequenza (notes,pitchnames,n_vocab)
  model = create_network (normalized_input,n_vocab,model_name,model)
  for i in range (len (nome_files)):
    prediction_output = genera_note(model, network_input, pitchnames, n_vocab,lunghezza)
    salva_in_midi (prediction_output,nome_files[i])
  if rm:
    return model

In [0]:
PATH = "drive/My Drive/path"
FORMATTAZIONE = 'T: {} epoch: {} brano: {}'
T = 0

In [0]:
def orderer (name):
  if name == '.ipynb_checkpoints':
    return -1
  n = int (name[2])
  name = name[12:]
  epoch = name[:3]
  if epoch[-1] not in '1234567890':
    epoch = epoch[:2]
  return int (epoch) + n*155

In [0]:
model = scrivi_brani (None,nome_files=['generazione'],rm = True)

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_1 (Activation)    (None, 256)              

In [0]:
for f in sorted(os.listdir(PATH),key = orderer):
  if f == '.ipynb_checkpoints':
    continue
  i = orderer (f)
  print (f'epoch: {i}')
  molti = not i%5
  files = [FORMATTAZIONE.format(T,i,j) for j in range (3)] if molti else [FORMATTAZIONE.format(T,i,0)]
  scrivi_brani (os.path.join (PATH,f),nome_files=files,model = model,lunghezza = 300)
  for f in files:
    zipObj = ZipFile('history.zip', 'a')
    zipObj.write(f+'.mid')
    zipObj.close()

epoch: 1
epoch: 1


/usr/lib/python3.6/zipfile.py:1378: UserWarning: Duplicate name: 'T: 0 epoch: 1 brano: 0.mid'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
epoch: 30
epoch: 31
epoch: 32
epoch: 33
epoch: 34
epoch: 35
epoch: 36
epoch: 37
epoch: 38
epoch: 39
epoch: 40
epoch: 43
epoch: 44
epoch: 45
epoch: 46
epoch: 47
epoch: 48
epoch: 49
epoch: 50
epoch: 51
epoch: 52
epoch: 53
epoch: 54
epoch: 55
epoch: 56
epoch: 57
epoch: 58
epoch: 59
epoch: 60
epoch: 61
epoch: 62
epoch: 63
epoch: 64
epoch: 65
epoch: 66
epoch: 67
epoch: 68
epoch: 69
epoch: 70
epoch: 71
epoch: 72
epoch: 73
epoch: 74
epoch: 75
epoch: 76
epoch: 77
epoch: 78
epoch: 79
epoch: 80
epoch: 81
epoch: 82
epoch: 83
epoch: 84
epoch: 85
epoch: 86
epoch: 87
epoch: 88
epoch: 89
epoch: 90
epoch: 91
epoch: 92
epoch: 93
epoch: 94
epoch: 96
epoch: 97
epoch: 98
epoch: 99
epoch: 100
epoch: 101
epoch: 103
epoch: 104
epoch: 105
epo

In [0]:
fd.download('history.zip')